1. What is the concept of an abstract superclass?

We can treat an abstract class as a superclass and extend it; its subclasses can override some or all of its inherited abstract methods. If through this overriding a subclass contains no more abstract methods, that class is concrete (and we can construct objects directly from it).

The purpose of an abstract class (often referred to as an ABC) is to provide an appropriate base class from which other classes can inherit. Abstract classes cannot be used to instantiate objects and serves only as an interface. Attempting to instantiate an object of an abstract class causes a compilation error.

2. What happens when a class statement&#39;s top level contains a basic assignment statement?

In computer programming, an assignment statement sets and/or re-sets the value stored in the storage location(s) denoted by a variable name; in other words, it copies a value into the variable. In most imperative programming languages, the assignment statement (or expression) is a fundamental construct.
Its purpose is saving the result of the expression to the right of the assignment operator to the variable on the left.

3. Why does a class need to manually call a superclass&#39;s __init__ method?

If you need something from super's __init__ to be done in addition to what is being done in the current class's __init__ , you must call it yourself, since that will not happen automatically"

It's incredible: he is wording exactly the contrary of the principle of inheritance.



It is not that "something from super's __init__ (...) will not happen automatically" , it is that it WOULD happen automatically, but it doesn't happen because the base-class' __init__ is overriden by the definition of the derived-clas __init__

So then, WHY defining a derived_class' __init__ , since it overrides what is aimed at when someone resorts to inheritance ??

It's because one needs to define something that is NOT done in the base-class' __init__ , and the only possibility to obtain that is to put its execution in a derived-class' __init__ function.
In other words, one needs something in base-class' __init__ in addition to what would be automatically done in the base-classe' __init__ if this latter wasn't overriden.
NOT the contrary.



Then, the problem is that the desired instructions present in the base-class' __init__ are no more activated at the moment of instantiation. In order to offset this inactivation, something special is required: calling explicitly the base-class' __init__ , in order to KEEP , NOT TO ADD, the initialization performed by the base-class' __init__ . That's exactly what is said in the official doc:

4. How can you augment, instead of completely replacing, an inherited method?

A class expression denotes a first-class value, just like a lambda expression:

(class superclass-expr decl-or-expr ...)
The superclass-expr determines the superclass for the new class. Each decl-or-expr is either a declaration related to methods, fields, and initialization arguments, or it is an expression that is evaluated each time that the class is instantiated. In other words, instead of a method-like constructor, a class has initialization expressions interleaved with field and method declarations.

By convention, class names end with %. The built-in root class is object%. The following expression creates a class with public methods get-size, grow, and eat:

(class object%
  (init size)                ; initialization argument
 
  (define current-size size) ; field
 
  (super-new)                ; superclass initialization
 
  (define/public (get-size)
    current-size)
 
  (define/public (grow amt)
    (set! current-size (+ amt current-size)))
 
  (define/public (eat other-fish)
    (grow (send other-fish get-size))))
The size initialization argument must be supplied via a named argument when instantiating the class through the new form:

(new (class object% (init size) ....) [size 10])

Of course, we can also name the class and its instance:

(define fish% (class object% (init size) ....))
(define charlie (new fish% [size 10]))
In the definition of fish%, current-size is a private field that starts out with the value of the size initialization argument. Initialization arguments like size are available only during class instantiation, so they cannot be referenced directly from a method. The current-size field, in contrast, is available to methods.

The (super-new) expression in fish% invokes the initialization of the superclass. In this case, the superclass is object%, which takes no initialization arguments and performs no work; super-new must be used, anyway, because a class must always invoke its superclass’s initialization.

Initialization arguments, field declarations, and expressions such as (super-new) can appear in any order within a class, and they can be interleaved with method declarations. The relative order of expressions in the class determines the order of evaluation during instantiation. For example, if a field’s initial value requires calling a method that works only after superclass initialization, then the field declaration must be placed after the super-new call. Ordering field and initialization declarations in this way helps avoid imperative assignment. The relative order of method declarations makes no difference for evaluation, because methods are fully defined before a class is instantiated.

13.1 Methods
Each of the three define/public declarations in fish% introduces a new method. The declaration uses the same syntax as a Racket function, but a method is not accessible as an independent function. A call to the grow method of a fish% object requires the send form:

> (send charlie grow 6)
> (send charlie get-size)
16

Within fish%, self methods can be called like functions, because the method names are in scope. For example, the eat method within fish% directly invokes the grow method. Within a class, attempting to use a method name in any way other than a method call results in a syntax error.

In some cases, a class must call methods that are supplied by the superclass but not overridden. In that case, the class can use send with this to access the method:

(define hungry-fish% (class fish% (super-new)
                       (define/public (eat-more fish1 fish2)
                         (send this eat fish1)
                         (send this eat fish2))))
 

Alternately, the class can declare the existence of a method using inherit, which brings the method name into scope for a direct call:

(define hungry-fish% (class fish% (super-new)
                       (inherit eat)
                       (define/public (eat-more fish1 fish2)
                         (eat fish1) (eat fish2))))
 

With the inherit declaration, if fish% had not provided an eat method, an error would be signaled in the evaluation of the class form for hungry-fish%. In contrast, with (send this ....), an error would not be signaled until the eat-more method is called and the send form is evaluated. For this reason, inherit is preferred.

Another drawback of send is that it is less efficient than inherit. Invocation of a method via send involves finding a method in the target object’s class at run time, making send comparable to an interface-based method call in Java. In contrast, inherit-based method invocations use an offset within the class’s method table that is computed when the class is created.

To achieve performance similar to inherit-based method calls when invoking a method from outside the method’s class, the programmer must use the generic form, which produces a class- and method-specific generic method to be invoked with send-generic:

(define get-fish-size (generic fish% get-size))
 

> (send-generic charlie get-fish-size)
16

> (send-generic (new hungry-fish% [size 32]) get-fish-size)
32

> (send-generic (new object%) get-fish-size)
generic:get-size: target is not an instance of the generic's

class

  target: (object)

  class name: fish%

Roughly speaking, the form translates the class and the external method name to a location in the class’s method table. As illustrated by the last example, sending through a generic method checks that its argument is an instance of the generic’s class.

Whether a method is called directly within a class, through a generic method, or through send, method overriding works in the usual way:

(define picky-fish% (class fish% (super-new)
                      (define/override (grow amt)
 
                        (super grow (* 3/4 amt)))))
(define daisy (new picky-fish% [size 20]))
 

> (send daisy eat charlie)
> (send daisy get-size)
32

The grow method in picky-fish% is declared with define/override instead of define/public, because grow is meant as an overriding declaration. If grow had been declared with define/public, an error would have been signaled when evaluating the class expression, because fish% already supplies grow.

Using define/override also allows the invocation of the overridden method via a super call. For example, the grow implementation in picky-fish% uses super to delegate to the superclass implementation.

13.2 Initialization Arguments
Since picky-fish% declares no initialization arguments, any initialization values supplied in (new picky-fish% ....) are propagated to the superclass initialization, i.e., to fish%. A subclass can supply additional initialization arguments for its superclass in a super-new call, and such initialization arguments take precedence over arguments supplied to new. For example, the following size-10-fish% class always generates fish of size 10:

(define size-10-fish% (class fish% (super-new [size 10])))
 

> (send (new size-10-fish%) get-size)
10

In the case of size-10-fish%, supplying a size initialization argument with new would result in an initialization error; because the size in super-new takes precedence, a size supplied to new would have no target declaration.

An initialization argument is optional if the class form declares a default value. For example, the following default-10-fish% class accepts a size initialization argument, but its value defaults to 10 if no value is supplied on instantiation:

(define default-10-fish% (class fish%
                           (init [size 10])
                           (super-new [size size])))
 

> (new default-10-fish%)
(object:default-10-fish% ...)

> (new default-10-fish% [size 20])
(object:default-10-fish% ...)

In this example, the super-new call propagates its own size value as the size initialization argument to the superclass.

5. How is the local scope of a class different from that of a function?

Local (or function) scope is the code block or body of any Python function or lambda expression. This Python scope contains the names that you define inside the function. These names will only be visible from the code of the function. It’s created at function call, not at function definition, so you’ll have as many different local scopes as function calls. This is true even if you call the same function multiple times, or recursively. Each call will result in a new local scope being created.
Functions: The Local Scope
The local scope or function scope is a Python scope created at function calls. Every time you call a function, you’re also creating a new local scope. On the other hand, you can think of each def statement and lambda expression as a blueprint for new local scopes. These local scopes will come into existence whenever you call the function at hand.

By default, parameters and names that you assign inside a function exist only within the function or local scope associated with the function call. When the function returns, the local scope is destroyed and the names are forgotten. Here’s how this works:

>>> def square(base):
...     result = base ** 2
...     print(f'The square of {base} is: {result}')
...
>>> square(10)
The square of 10 is: 100
>>> result  # Isn't accessible from outside square()
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
    result
NameError: name 'result' is not defined
>>> base  # Isn't accessible from outside square()
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
    base
NameError: name 'base' is not defined
>>> square(20)
The square of 20 is: 400
square() is a function that computes the square of a given number, base. When you call the function, Python creates a local scope containing the names base (an argument) and result (a local variable). After the first call to square(), base holds a value of 10 and result holds a value of 100. The second time, the local names will not remember the values that were stored in them the first time the function was called. Notice that base now holds the value 20 and result holds 400.

Note: If you try to access result or base after the function call, then you get a NameError, because these only exist in the local scope created by the call to square(). Whenever you try to access a name that isn’t defined in any Python scope, you’ll get a NameError. The error message will include the name that couldn’t be found.

Since you can’t access local names from statements that are outside the function, different functions can define objects with the same name. Check out this example: